# このJupyter Notebookの使い方
これはレシピ集として作成されています。最初に、初期設定セクションにて利用するElasticsearch環境やOpenAI環境、HuggingFace環境の接続情報を設定します。その後は、実行したいセクションA. B. C. ..から始めて順番にコマンドを実行してください。
そのためにセクション間で重複したコードが繰り返しあります。

# 1. 初期設定

## ライブラリの有効化

In [ ]:
%pip install -q elasticsearch langchain openai tiktoken

In [213]:
from pprint import pprint
import os
from getpass import getpass
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk

## Elasticsearchの設定

In [178]:
ELASTIC_API_KEY = getpass("Elastic deployment API Key")
ELASTIC_CLOUD_ID = getpass("Elastic deployment Cloud ID")
ELASTIC_URL = getpass("Elastic deployment URL. No need if Cloud ID is provided.")
ELASTIC_USER = getpass("Elastic user. No need if API key is provided.")
ELASTIC_PASSWORD = getpass("Elastic password. No need if API key is provided.")

if ELASTIC_CLOUD_ID != '' and ELASTIC_API_KEY != '':
  es = Elasticsearch(
    cloud_id=ELASTIC_CLOUD_ID,
    api_key=ELASTIC_API_KEY,
    request_timeout=300
  )
elif ELASTIC_URL != '' and ELASTIC_USER != '' and ELASTIC_PASSWORD != '':
  es = Elasticsearch(
    hosts = ELASTIC_URL,
    basic_auth=(ELASTIC_USER, ELASTIC_PASSWORD),
    request_timeout=300
  )
elif ELASTIC_URL != '' and ELASTIC_USER == '':
  es = Elasticsearch(
    hosts = ELASTIC_URL,
    # request_timeout=300,
    request_timeout=300
  )
else:
  print("env needs to set either ELASTIC_CLOUD_ID or ELASTIC_URL")


pprint(es.info()) # should return cluster info

ObjectApiResponse({'name': 'instance-0000000024', 'cluster_name': '507a2cf6ba204071943512e0537eee58', 'cluster_uuid': 'oF-xDLtXRCet87gRuM3eJg', 'version': {'number': '8.10.2', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '6d20dd8ce62365be9b1aca96427de4622e970e9e', 'build_date': '2023-09-19T08:16:24.564900370Z', 'build_snapshot': False, 'lucene_version': '9.7.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})


## OpenAI APIの設定

In [179]:
os.environ["OPENAI_API_KEY"] = getpass("OpenAI API key: ")

## HuggingFace APIの設定

In [261]:
huggingface_api_Key = getpass("Enter your HF Inference API Key:\n\n")

## Elastic 上のMLモデルの設定

In [181]:
%pip install -q eland elasticsearch transformers sentence_transformers

Note: you may need to restart the kernel to use updated packages.


In [182]:
import tempfile
from eland.ml.pytorch import PyTorchModel
from eland.ml.pytorch.transformers import TransformerModel

In [183]:
def load_model(es_connection, model_id, task_type):
  with tempfile.TemporaryDirectory() as tmp_dir:
    print(f"Loading HuggingFace transformer tokenizer and model [{model_id}] for task [{task_type}]" )

    tm = TransformerModel(model_id=model_id, task_type=task_type)
    model_path, config, vocab_path = tm.save(tmp_dir)

    ptm = PyTorchModel(es_connection, tm.elasticsearch_model_id())
    model_exists = es_connection.options(ignore_status=404).ml.get_trained_models(model_id=ptm.model_id).meta.status == 200

    if model_exists:
      print("Model has already been imported")
    else:
      print("Importing model")
      ptm.import_model(model_path=model_path, config_path=None, vocab_path=vocab_path, config=config)
      print(f"Model successfully imported with id '{ptm.model_id}'")

def start_model(es_connection, model_id, task_type):
    tm = TransformerModel(model_id=model_id, task_type=task_type)
    ptm = PyTorchModel(es_connection, tm.elasticsearch_model_id())
    print("Starting model deployment")
    ptm.start()
    print(f"Model successfully started with id '{ptm.model_id}'")

def stop_model(es_connection, model_id, task_type):
    tm = TransformerModel(model_id=model_id, task_type=task_type)
    ptm = PyTorchModel(es_connection, tm.elasticsearch_model_id())
    ptm.stop()


### Elastic ML E5モデルのロード

In [184]:
es.info()
load_model(es, "intfloat/multilingual-e5-base", "text_embedding")

Loading HuggingFace transformer tokenizer and model [intfloat/multilingual-e5-base] for task [text_embedding]
Model has already been imported


### Elastic ML E5モデルの開始

In [187]:
print(es.info())
start_model(es, "intfloat/multilingual-e5-base", "text_embedding")

{'name': 'instance-0000000024', 'cluster_name': '507a2cf6ba204071943512e0537eee58', 'cluster_uuid': 'oF-xDLtXRCet87gRuM3eJg', 'version': {'number': '8.10.2', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '6d20dd8ce62365be9b1aca96427de4622e970e9e', 'build_date': '2023-09-19T08:16:24.564900370Z', 'build_snapshot': False, 'lucene_version': '9.7.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'}
Starting model deployment
Model successfully started with id 'intfloat__multilingual-e5-base'


### (Elastic ML E5モデルの停止)

In [186]:
es.info()
stop_model(es, "intfloat/multilingual-e5-base", "text_embedding")

/Users/nobuhikosekiya/lab/projects/elasticsearch-langchain-cookbook/.venv/lib/python3.10/site-packages/eland/ml/pytorch/_pytorch_model.py:152: ElasticsearchWarning: The default [remove_binary] value of 'false' is deprecated and will be set to 'true' in a future release. Set [remove_binary] explicitly to 'true' or 'false' to ensure no behavior change.
  self._client.ml.stop_trained_model_deployment(model_id=self.model_id)


## 検索対象の設定


In [ ]:
urls = [
    #検索分析を使ってEコマースソリューションを強化する
    "https://www.elastic.co/jp/blog/using-search-analytics-to-strengthen-ecommerce-solutions",

    #Elasticで、個々のお客様に応じたEコマース向け検索エクスペリエンスを構築
    "https://www.elastic.co/jp/blog/building-personalized-ecommerce-search-experiences-with-elastic",

    #Elastic App SearchでSpring Bootを使用する
    "https://www.elastic.co/jp/blog/using-spring-boot-with-elastic-app-search",

    #Elasticエンタープライズサーチでeコマースの競争力を高める6つの方法
    "https://www.elastic.co/jp/blog/6-ways-elastic-enterprise-search-creates-a-competitive-edge-in-ecommerce",

    #Elasticを使用して検索エクスペリエンスをパーソナライズする方法
    "https://www.elastic.co/jp/blog/how-to-personalize-search-experiences-using-elastic",

    #検索候補の絞り込みと検索クエリの修正を実装する方法
    "https://www.elastic.co/jp/blog/how-to-build-scoped-search-suggestions-and-search-query-corrections",

    #画像の類似検索の5つの技術要素
    "https://www.elastic.co/jp/blog/5-technical-components-image-similarity-search",

    #Elasticで画像の類似検索を実装する方法
    "https://www.elastic.co/jp/blog/implement-image-similarity-search-elastic",

    #Elasticsearchでの画像の類似検索の概要
    "https://www.elastic.co/jp/blog/overview-image-similarity-search-in-elastic",
]

## テストしたい質問の設定

In [218]:
questions = [
    "画像検索の方法について知りたい",
    "検索速度を速くする方法について知りたい",
    "Webサイトに検索を設けるにはどうすればいいか"
]

## 結果ホルダー

In [368]:
import json

# Initialize a list to store the results
result_list = []

# Define the file path for data persistence
file_path = 'results.json'

def add_result(query, search_logic, result):
    # Check if a result with the same 'query' and 'search_logic' exists
    for item in result_list:
        if item['query'] == query and item['search_logic'] == search_logic:
            # Update the existing result
            item['result'] = result
            break
    else:
        # If no existing result found, add a new one
        result_list.append({'query': query, 'search_logic': search_logic, 'result': result})
    
    # Save the results to the file
    save_results()

def save_results():
    with open(file_path, 'w') as file:
        json.dump(result_list, file, ensure_ascii=False, indent=2)


def load_results():
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            data = json.load(file)
            result_list.clear()
            result_list.extend(data)
    except FileNotFoundError:
        # If the file doesn't exist, start with an empty results list
        result_list.clear()

# Load existing results from the file (if any)
load_results()

# Results after adding the examples
print(result_list)


[{'query': '画像検索の方法について知りたい', 'search_logic': 'bm25', 'result': {'query': '画像検索の方法について知りたい', 'result': ['{"metadata": {"source": "https://www.elastic.co/jp/blog/6-ways-elastic-enterprise-search-creates-a-competitive-edge-in-ecommerce"}, "highlight": {"page_content.kuromoji": ["Elasticエンタープライズサーチでeコマースの競争力を高める6つの<em>方法</em> Elastic Cloudの無料トライアルに登録 お使いの<em>検索</em>アプリケーションは、皆さんが思う以上にパワフルです。", "今回ご紹介する機能を使えば、<em>検索</em>データを活用して優れた顧客エクスペリエンスを構築できます。", "「たとえば、皆さんが会議に加わって『ユーザーがクエリXの後に何を<em>検索</em>しているか<em>知り</em>たい』と言ったときに、ユーザーが<em>検索</em>するキーワードのシーケンスを作成できます。", "コンバージョンと、コンバージョンに至る前の<em>検索</em>で区切ることも可能です」 <em>検索</em>と発見は重要ですが、独立して運用されているわけではありません。", "バックログには膨大な数の商品があり、それを最初の<em>検索</em>結果に表示させたい場合もあるでしょう。"]}}', '{"metadata": {"source": "https://www.elastic.co/jp/blog/5-technical-components-image-similarity-search"}, "highlight": {"page_content.kuromoji": ["<em>画像</em>の類似<em>検索</em>の5つの技術要素 AI時代の<em>検索</em>ツールキット 推論：ユーザーのクエリをベクトル化する  <em>検索</em>：類似<em>画像</em> Elastic Cloudの無料トライアルに登録 著者 Rado

# 2. 検索対象のドキュメントをダウンロードして整形

In [188]:
%pip install -qU pytest-playwright beautifulsoup4
!playwright install

Note: you may need to restart the kernel to use updated packages.
zsh:1: /Users/nobuhikosekiya/lab/projects/elasticsearch-langchain-cookbook/.venv/bin/playwright: bad interpreter: /Users/nobuhikosekiya/lab/projects/es-langchain/.venv/bin/python: no such file or directory


## 2-A. WebBaseLoaderでWebページのテキストを抽出したDocumentを作る
このセクションでは、指定のWebページに含まれる全テキストを単純抽出して、Documentとして格納します。
結果は**docs**のリストに格納されます。

In [190]:
from langchain.document_loaders import WebBaseLoader

loader = WebBaseLoader(urls)
docs = loader.load()
pprint(docs, indent=4, width=30)

[   Document(page_content='検索分析にElasticを使ってEコマースソリューションを強化する | Elastic BlogSkip to main contentPlatformPlatformソリューションソリューション導入事例導入事例リソースリソース料金料金ドキュメントドキュメントLanguage pickerDeutschEnglishEspañolFrançais日本語한국어简体中文PortuguêsSearchLogin無料トライアルを始めるセールスへのお問い合わせElasticsearch Platformオブザーバビリティも、セキュリティも、検索ソリューションも、Elasticsearchプラットフォームならすべて実現できます。Elasticの概要ELK Stackデータインジェストから検索、分析、可視化まで、すべてが思いのままKibanaElasticsearch統合機能群ELK Stackの概要Elastic Cloudお好みのクラウドプロバイダーでElasticを活用し、必要な答えを見つけられます。Cloudの概要パートナーセールスへのお問い合わせオブザーバビリティアプリやインフラをまとめて可視化し、問題を先回りして修正できます。ログ監視アプリケーションパフォーマンス監視インフラ監視シンセティック監視リアルユーザー監視ユニバーサルプロファイリングAIOpsOpenTelemetryオブザーバビリティの概要セキュリティサイバー脅威を広範囲で防御、調査し迅速に対応できます。継続的な監視脅威ハンティング調査とインシデントレスポンス脅威保護の自動化セキュリティの概要Searchあらゆるクラウドで迅速に検索結果を示し、パーソナライゼーションも高められます。生成的AI検索アプリEコマースWebサイト社内コンテンツ検索カスタマーサポートSearchの概要業界別公的セクター金融サービス通信医療テクノロジー小売＆eコマース製造＆自動車すべての業界を表示ソリューション別次に取るべき行動を大規模かつ柔軟、スピーディに見つけられます。オブザーバビリティセキュリティSearchお客様事例Cisco社：AIを導入して検索エクスペリエンスを変革続きを読むRWE社：再生可能エネルギー取引事業を強化続きを読むComcast社：エンジニアリングのスピード

## 2-B. ページ内の不要な情報を削ぎ落としたDocumentを作成する
ここでは、以下の方法でメインのコンテンツだけを抽出するようにしています。
1. メインのコンテンツが含まれるHTMLのmainタグを抽出します。
2. 不要なscriptタグを除去しています。
3. さらにmainの中の必要なタグ h1, h3, h4, p, ulなどに絞ってテキストを抽出します。

結果のWebページのテキストが含まれたDocumentは**docs**のリストに格納されます。

In [246]:
from langchain.document_loaders import AsyncHtmlLoader
from bs4 import BeautifulSoup
from pprint import pprint

def extract_main_content(html_content):
    soup = BeautifulSoup(html_content, "html.parser")
    main_html = soup.find("main")
    scripts = main_html.find_all('script')
    for s in scripts:
        s.extract()

    if main_html:
        return main_html
    return None

loader = AsyncHtmlLoader(urls) # Use to load raw html into page_content
html_docs = loader.load()

for html_doc in html_docs:
#     pprint(html_doc.metadata)
    # print("======= 抽出前 =======")
    # pprint(html_doc.page_content[:1000])
    # print("======= 抽出後 =======")
    extraced_html = extract_main_content(f"<html><body>{html_doc.page_content}</body></html>")
    html_doc.page_content = f"<html><body>{extraced_html}</body></html>"
    # pprint(html_doc.page_content[:1000])
    # print("==============")
pprint(html_docs)

Fetching pages: 100%|##########| 9/9 [00:01<00:00,  8.20it/s]


[Document(page_content='<html><body><main id="main-content" role="main"><div class="jsx-1686534253 blog-post-detail"><div class="jsx-1686534253 container mt-6"><div class="jsx-1686534253 row justify-content-center" id="sideTable"><div class="jsx-1686534253 col-12 col-lg-8"><div class="jsx-1686534253 main-header-block"><div class="jsx-1686534253 section" id="検索分析を使ってeコマースソリューションを強化する"><div class="jsx-1955866259 title-wrapper"><a class="jsx-1955866259" id="検索分析を使ってeコマースソリューションを強化する"></a><h1 class="jsx-1955866259" tag="H1">検索分析を使ってEコマースソリューションを強化する</h1></div></div><div class="jsx-1686534253 author-block mt-6"><div class="jsx-1686534253 author-desc d-flex"><div class="jsx-1686534253 d-flex flex-wrap mb-2"><p class="jsx-1686534253 medium">著者</p><a class="author-name" href="/blog/author/-the-elastic-enterprise-search-team"><p class="jsx-1686534253 medium"> The Elastic Enterprise Search team</p></a></div><time datetime="1660089600000">2022年8月10日</time></div><div class="jsx-1686534253 toc-btn"

In [247]:
from langchain.document_transformers import BeautifulSoupTransformer
bs_transformer = BeautifulSoupTransformer()

for html_doc in html_docs:
    extracted_text = bs_transformer.extract_tags(html_content=html_doc.page_content, tags=["h1","h3","h4","p","ul"])
    # pprint(extracted_text)
    html_doc.page_content = extracted_text
# docs_transformed = bs_transformer.transform_documents(docs, tags_to_extract=["a"])
# docs_transformed

docs = html_docs
pprint(docs)

[Document(page_content='検索分析を使ってEコマースソリューションを強化する Eコマース企業のデータ可視化例 Elastic Cloudの無料トライアルに登録 著者  The Elastic Enterprise Search team Twitter リンクトイン Facebook メール 印刷 お客様にすばらしい体験を届けるためには、自社が使えるツールのすべてをしっかりと理解しておくことが重要です。特に、現在のように経済が不安定な時期に少しでも優位に立ちたいと思うのであれば、その重要性はいっそう大きくなると言えます。良質なカスタマーエクスペリエンスを提供できれば、ページを閲覧した人を商品の購入に導き、長期的な顧客に変えていくことができます。それこそが、他社に対して優位を得るための第一歩にほかなりません。  お客様に最適な体験を届けるために必要なのは、ビジネスデータだけではありません。自社のEコマースプラットフォームから集めた検索分析データも非常に重要です。お客様がサイトで検索機能を利用するたびに、お客様のニーズを把握するうえで重要なデータが供給されています。Elasticなら、そのようなデータからさまざまなインサイトを簡単に導き出せるため、製品ミックスの改善や、セールスの最適化に役立ちます。 お客様がサイトで製品やサービスを検索したまさにそのとき、サイトにはさまざまな情報が届いています。ユーザーエクスペリエンス自体についてだけでなく、その質についてもフィードバックが届いているのです。そのようなフィードバックを念頭に置いたソリューションを構築すれば、ユーザーに好印象を与え、エンゲージメントを高めることが可能になります。しかし、検索分析がもたらすインサイトは現在、十分に活用されているとは言えません。そのため、これをうまく利用していくことが、優れたユーザーエクスペリエンスの実現に向けた鍵となります。  重要なデータポイントの1つに、「検索回数が上位のクエリ」があります。上位のクエリを見ると、多くのお客様が強い関心を抱いている品目や、流行している品目が直接明らかになることがあるからです。このデータは特に、ユーザーの地理情報その他のプロファイル情報など、各種メタデータと合わせて検討すると有用な情報に変わります。たとえば、運営しているEコマースサイトの

# 3. 様々なサーチを試そう

## 3-A. Elasticsearchのキーワード検索 (BM25)

### インデックス作成
Kuromojiをアナライザーとして設定したElasticsearchのインデックスを作成します。

In [196]:
INDEX_NAME="test_index_bm25"

In [197]:

if es.indices.exists(index=INDEX_NAME):
    # If it exists, delete the index
    es.indices.delete(index=INDEX_NAME)
    print(f"Index '{INDEX_NAME}' deleted successfully.")
else:
    print(f"Index '{INDEX_NAME}' does not exist.")

es.indices.create(
  index=INDEX_NAME,
  settings={
      "index": {
          "number_of_shards": 1,
          "number_of_replicas": 0
      }
  }
)


Index 'test_index_bm25' deleted successfully.


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'test_index_bm25'})

In [198]:
es.indices.close(index=INDEX_NAME)

add_settings = {
  "index": {
    "analysis": {
      "char_filter": {
        "normalize": {
          "mode": "compose",
          "name": "nfkc",
          "type": "icu_normalizer"
        }
      }
    }
  }
}
es.indices.put_settings(index=INDEX_NAME, body=add_settings)

add_settings = {
  "index": {
    "analysis": {
      "filter": {
        "ja_index_synonym": {
          "type": "synonym",
          "lenient": "false",
          "synonyms": []
        }
      }
    }
  }
}
es.indices.put_settings(index=INDEX_NAME, body=add_settings)

add_settings = {
  "index": {
    "analysis": {
      "tokenizer": {
        "ja_kuromoji_tokenizer": {
          "mode": "search",
          "discard_compound_token": "true",
          "type": "kuromoji_tokenizer"
        }
      }
    }
  }
}

es.indices.put_settings(index=INDEX_NAME, body=add_settings)

# Define a synonym set
synonyms_set = [
    # {
    #   "id": "test-1",
    #   "synonyms": "foo, bar"
    # },
    # {
    #   "id": "test-2",
    #   "synonyms": "test => check"
    # }
  ]

# Define the identifier for the synonym set
synonym_id = "ja_search_synonym-set"

from elasticsearch.client import SynonymsClient
# Call the put_synonym function to add or update the synonym set
response = SynonymsClient(es).put_synonym(id=synonym_id, synonyms_set=synonyms_set)

# Define the new settings you want to apply
add_settings = {
  "index": {
    "analysis": {
      "filter": {
        "ja_search_synonym": {
          "type": "synonym_graph",
          "synonyms_set": "ja_search_synonym-set",
          "updateable": "true"
        }
      },
      "analyzer": {
        "ja_kuromoji_index_analyzer": {
          "filter": [
            "kuromoji_baseform",
            "kuromoji_part_of_speech",
            "ja_index_synonym",
            "cjk_width",
            "ja_stop",
            "kuromoji_stemmer",
            "lowercase"
          ],
          "char_filter": [
            "normalize"
          ],
          "type": "custom",
          "tokenizer": "ja_kuromoji_tokenizer"
        },
        "ja_kuromoji_search_analyzer": {
          "filter": [
            "kuromoji_baseform",
            "kuromoji_part_of_speech",
            "ja_search_synonym",
            "cjk_width",
            "ja_stop",
            "kuromoji_stemmer",
            "lowercase"
          ],
          "char_filter": [
            "normalize"
          ],
          "type": "custom",
          "tokenizer": "ja_kuromoji_tokenizer"
        }
      }
    }
  }
}

es.indices.put_settings(index=INDEX_NAME, body=add_settings)

es.indices.open(index=INDEX_NAME, request_timeout=60)


/var/folders/tg/fg_nnncd3p7__vrf94g5k56m0000gn/T/ipykernel_9368/1678612302.py:119: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  es.indices.open(index=INDEX_NAME, request_timeout=60)


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True})

In [199]:
es.indices.close(index=INDEX_NAME, request_timeout=60)

# Define the new mapping
add_mapping = {
  "properties": {
    "page_content": {
      "type": "text",
      "fields": {
        "kuromoji": {
          "type": "text",
          "analyzer": "ja_kuromoji_index_analyzer",
          "search_analyzer": "ja_kuromoji_search_analyzer"
        }
      }
    },
    "metadata": {
      "type": "object",
      "properties": {
        "source": {
          "type": "keyword"
        }
      }
    }
  }
}

es.indices.put_mapping(index=INDEX_NAME, body=add_mapping)

es.indices.open(index=INDEX_NAME)

/var/folders/tg/fg_nnncd3p7__vrf94g5k56m0000gn/T/ipykernel_9368/2817861892.py:1: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  es.indices.close(index=INDEX_NAME, request_timeout=60)


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True})

### インジェスト

In [277]:
from elasticsearch import Elasticsearch, helpers

# 前の実行で残っているDocumentはクリアしてからインジェストします
if es.indices.exists(index=INDEX_NAME):
    es.delete_by_query(index=INDEX_NAME, body={"query": {"match_all": {}}})

docs_json = [doc.to_json()["kwargs"] for doc in docs]
# print(docs_json)

index_docs = []
for doc_json in docs_json:
    index_docs.append({
        "_index": INDEX_NAME,
        "_source": doc_json,
    })

helpers.bulk(es, index_docs)

response = es.search(index=INDEX_NAME, query={"match_all": {}}, fields=["metadata"])
for hit in response['hits']['hits']:
    metadata = hit['_source']['metadata']
    print(f"metadata: {metadata}")

metadata: {'source': 'https://www.elastic.co/jp/blog/using-search-analytics-to-strengthen-ecommerce-solutions'}
metadata: {'source': 'https://www.elastic.co/jp/blog/building-personalized-ecommerce-search-experiences-with-elastic'}
metadata: {'source': 'https://www.elastic.co/jp/blog/using-spring-boot-with-elastic-app-search'}
metadata: {'source': 'https://www.elastic.co/jp/blog/6-ways-elastic-enterprise-search-creates-a-competitive-edge-in-ecommerce'}
metadata: {'source': 'https://www.elastic.co/jp/blog/how-to-personalize-search-experiences-using-elastic'}
metadata: {'source': 'https://www.elastic.co/jp/blog/how-to-build-scoped-search-suggestions-and-search-query-corrections'}
metadata: {'source': 'https://www.elastic.co/jp/blog/5-technical-components-image-similarity-search'}
metadata: {'source': 'https://www.elastic.co/jp/blog/implement-image-similarity-search-elastic'}
metadata: {'source': 'https://www.elastic.co/jp/blog/overview-image-similarity-search-in-elastic'}


### サーチ

In [365]:
def do_search(query_text):
    query = {
        "bool": {
            "must": [
                {
                    "match": {
                        "page_content.kuromoji": {
                            "query": query_text,
                            "analyzer": "ja_kuromoji_search_analyzer"
                        }
                    }
                }
            ]
        }
    }
    highlight = {
        "fields": {
            "page_content.kuromoji": {}
        }
    }
    fields = ["metadata"]
    response = es.search(index=INDEX_NAME, query=query, fields=fields, highlight=highlight)

    results = []
    # Iterate through the search results and access the fields you want
    for hit in response['hits']['hits']:
        metadata = hit['_source']['metadata']
        score = hit['_score']
        highlight = hit['highlight']
        print(f"_score: {score}, metadata: {metadata}")
        pprint(f"{highlight}")
        r = {
            "metadata": metadata,
            "highlight": highlight
        }
        results.append(json.dumps(r, ensure_ascii=False))

    return {"query": query_text, "result": results}

In [366]:
for query in questions:
    print(query)
    result = do_search(query)
    add_result(search_logic="bm25", query=query, result=result)

画像検索の方法について知りたい
_score: 2.9904552, metadata: {'source': 'https://www.elastic.co/jp/blog/6-ways-elastic-enterprise-search-creates-a-competitive-edge-in-ecommerce'}
("{'page_content.kuromoji': ['Elasticエンタープライズサーチでeコマースの競争力を高める6つの<em>方法</em> "
 "Elastic Cloudの無料トライアルに登録 お使いの<em>検索</em>アプリケーションは、皆さんが思う以上にパワフルです。', "
 "'今回ご紹介する機能を使えば、<em>検索</em>データを活用して優れた顧客エクスペリエンスを構築できます。', "
 "'「たとえば、皆さんが会議に加わって『ユーザーがクエリXの後に何を<em>検索</em>しているか<em>知り</em>たい』と言ったときに、ユーザーが<em>検索</em>するキーワードのシーケンスを作成できます。', "
 "'コンバージョンと、コンバージョンに至る前の<em>検索</em>で区切ることも可能です」 "
 "<em>検索</em>と発見は重要ですが、独立して運用されているわけではありません。', "
 "'バックログには膨大な数の商品があり、それを最初の<em>検索</em>結果に表示させたい場合もあるでしょう。']}")
_score: 1.9122819, metadata: {'source': 'https://www.elastic.co/jp/blog/5-technical-components-image-similarity-search'}
("{'page_content.kuromoji': ['<em>画像</em>の類似<em>検索</em>の5つの技術要素 "
 'AI時代の<em>検索</em>ツールキット 推論：ユーザーのクエリをベクトル化する  <em>検索</em>：類似<em>画像</em> '
 "Elastic Cloudの無料トライアルに登録 著者 Radovan Ondas,', "
 "'このような種類の類似<em>検索</em>を実装するには、テキスト

## 3-B. （Open AI Embeddingを利用） Elasticsearchのセマンティック検索
LangchainのElasticsearchStoreというクラスを使って色々なAI/MLモデルを切り替えながらElasticsearchのベクトル検索をテストしていくことができます。

参考：https://python.langchain.com/docs/integrations/vectorstores/elasticsearch

### インジェスト
補足：ElasticsearchStoreの中で自動的にインデックスの作成も行われています。

In [267]:
from langchain.vectorstores.elasticsearch import ElasticsearchStore
from langchain.embeddings.openai import OpenAIEmbeddings

embedding = OpenAIEmbeddings()

db_openai = ElasticsearchStore(
    es_connection=es,
    index_name="test_index_openai",
    embedding=embedding
)

In [268]:
# 前の実行で残っているDocumentはクリアしてからインジェストします
if es.indices.exists(index="test_index_openai"):
    db_openai.client.delete_by_query(index="test_index_openai", body={"query": {"match_all": {}}})
db_openai.add_documents(docs)
db_openai.client.indices.refresh(index="test_index_openai")

ObjectApiResponse({'_shards': {'total': 2, 'successful': 2, 'failed': 0}})

### サーチ

In [325]:
for query in questions:
    print(query)
    results = db_openai.similarity_search(query)
    [print(element.metadata) for element in results]
    add_result(search_logic="vector_openai_embeddings", query=query, result=[element.metadata for element in results])

画像検索の方法について知りたい
{'source': 'https://www.elastic.co/jp/blog/overview-image-similarity-search-in-elastic'}
{'source': 'https://www.elastic.co/jp/blog/implement-image-similarity-search-elastic'}
{'source': 'https://www.elastic.co/jp/blog/5-technical-components-image-similarity-search'}
{'source': 'https://www.elastic.co/jp/blog/how-to-build-scoped-search-suggestions-and-search-query-corrections'}
検索速度を速くする方法について知りたい
{'source': 'https://www.elastic.co/jp/blog/how-to-build-scoped-search-suggestions-and-search-query-corrections'}
{'source': 'https://www.elastic.co/jp/blog/6-ways-elastic-enterprise-search-creates-a-competitive-edge-in-ecommerce'}
{'source': 'https://www.elastic.co/jp/blog/using-search-analytics-to-strengthen-ecommerce-solutions'}
{'source': 'https://www.elastic.co/jp/blog/building-personalized-ecommerce-search-experiences-with-elastic'}
Webサイトに検索を設けるにはどうすればいいか
{'source': 'https://www.elastic.co/jp/blog/how-to-build-scoped-search-suggestions-and-search-query-corrections'}
{'so

## 3-C. (HuggingFaceにあるE5モデルを利用） Elasticsearchのセマンティック検索

#### インジェスト

In [262]:
from langchain.vectorstores.elasticsearch import ElasticsearchStore
from langchain.embeddings import HuggingFaceInferenceAPIEmbeddings

MODEL_ID="intfloat/multilingual-e5-base"

if not huggingface_api_Key:
    raise Exception("huggingface_api_Key is not set")

embedding = HuggingFaceInferenceAPIEmbeddings(
    api_key=huggingface_api_Key,
    model_name=MODEL_ID
)
db_huggingface_e5 = ElasticsearchStore(
    es_connection=es,
    index_name="test_index_huggingface_e5",
    embedding=embedding
)


In [263]:
if db_huggingface_e5.client.indices.exists(index="test_index_huggingface_e5"):
    db_huggingface_e5.client.delete_by_query(index="test_index_huggingface_e5", body={"query": {"match_all": {}}})
db_huggingface_e5.add_documents(docs)
db_huggingface_e5.client.indices.refresh(index="test_index_huggingface_e5")

ObjectApiResponse({'_shards': {'total': 2, 'successful': 2, 'failed': 0}})

#### サーチ

In [326]:
for query in questions:
    print(query)
    results = db_huggingface_e5.similarity_search(query)
    [print(element.metadata) for element in results]
    add_result(search_logic="vector_huggingface_e5", query=query, result=[element.metadata for element in results])

画像検索の方法について知りたい
{'source': 'https://www.elastic.co/jp/blog/overview-image-similarity-search-in-elastic'}
{'source': 'https://www.elastic.co/jp/blog/5-technical-components-image-similarity-search'}
{'source': 'https://www.elastic.co/jp/blog/implement-image-similarity-search-elastic'}
{'source': 'https://www.elastic.co/jp/blog/how-to-build-scoped-search-suggestions-and-search-query-corrections'}
検索速度を速くする方法について知りたい
{'source': 'https://www.elastic.co/jp/blog/how-to-build-scoped-search-suggestions-and-search-query-corrections'}
{'source': 'https://www.elastic.co/jp/blog/overview-image-similarity-search-in-elastic'}
{'source': 'https://www.elastic.co/jp/blog/how-to-personalize-search-experiences-using-elastic'}
{'source': 'https://www.elastic.co/jp/blog/6-ways-elastic-enterprise-search-creates-a-competitive-edge-in-ecommerce'}
Webサイトに検索を設けるにはどうすればいいか
{'source': 'https://www.elastic.co/jp/blog/how-to-build-scoped-search-suggestions-and-search-query-corrections'}
{'source': 'https://www.elast

## 3-D. (ElasticsearchにアップしたE5モデルを利用） Elasticsearchのセマンティック検索

### インジェスト

In [225]:
from langchain.vectorstores.elasticsearch import ElasticsearchStore
from langchain.embeddings.elasticsearch import ElasticsearchEmbeddings

MODEL_ID="intfloat__multilingual-e5-base"

embedding = ElasticsearchEmbeddings.from_es_connection(
    es_connection=es,
    model_id=MODEL_ID
)

db_esml_e5 = ElasticsearchStore(
    es_connection=es,
    index_name="test_index_esml_e5",
    embedding=embedding,
    strategy=ElasticsearchStore.ApproxRetrievalStrategy()
)

In [271]:
if db_esml_e5.client.indices.exists(index="test_index_esml_e5"):
    db_esml_e5.client.delete_by_query(index="test_index_esml_e5", body={"query": {"match_all": {}}})
print(len(docs))
db_esml_e5.add_documents(docs)
db_esml_e5.client.indices.refresh(index="test_index_esml_e5")

9


ObjectApiResponse({'_shards': {'total': 2, 'successful': 2, 'failed': 0}})

### サーチ

In [327]:
for query in questions:
    print(query)
    results = db_esml_e5.similarity_search(query)
    [print(element.metadata) for element in results]
    add_result(search_logic="vector_elastic_e5", query=query, result=[element.metadata for element in results])

画像検索の方法について知りたい
{'source': 'https://www.elastic.co/jp/blog/5-technical-components-image-similarity-search'}
{'source': 'https://www.elastic.co/jp/blog/overview-image-similarity-search-in-elastic'}
{'source': 'https://www.elastic.co/jp/blog/using-search-analytics-to-strengthen-ecommerce-solutions'}
{'source': 'https://www.elastic.co/jp/blog/implement-image-similarity-search-elastic'}
検索速度を速くする方法について知りたい
{'source': 'https://www.elastic.co/jp/blog/how-to-personalize-search-experiences-using-elastic'}
{'source': 'https://www.elastic.co/jp/blog/5-technical-components-image-similarity-search'}
{'source': 'https://www.elastic.co/jp/blog/how-to-build-scoped-search-suggestions-and-search-query-corrections'}
{'source': 'https://www.elastic.co/jp/blog/using-search-analytics-to-strengthen-ecommerce-solutions'}
Webサイトに検索を設けるにはどうすればいいか
{'source': 'https://www.elastic.co/jp/blog/how-to-personalize-search-experiences-using-elastic'}
{'source': 'https://www.elastic.co/jp/blog/using-search-analytics-to-

## 3-E. Elasticsearch RRF ハイブリッド検索

### インジェスト

In [228]:
from langchain.vectorstores.elasticsearch import ElasticsearchStore
from langchain.embeddings.elasticsearch import ElasticsearchEmbeddings

MODEL_ID="intfloat__multilingual-e5-base"

embedding = ElasticsearchEmbeddings.from_es_connection(
    es_connection=es,
    model_id=MODEL_ID
)

db_esml_e5_hybrid = ElasticsearchStore(
    es_connection=es.options(request_timeout=3600),
    index_name="test_index_esml_e5_hybrid",
    embedding=embedding,
    strategy=ElasticsearchStore.ApproxRetrievalStrategy(hybrid=True)
)

In [273]:
if db_esml_e5_hybrid.client.indices.exists(index="test_index_esml_e5_hybrid"):
    db_esml_e5_hybrid.client.delete_by_query(index="test_index_esml_e5_hybrid", body={"query": {"match_all": {}}})
print(len(docs))
db_esml_e5_hybrid.add_documents(docs)
db_esml_e5_hybrid.client.indices.refresh(index="test_index_esml_e5_hybrid")

9


ObjectApiResponse({'_shards': {'total': 2, 'successful': 2, 'failed': 0}})

### サーチ

In [328]:
for query in questions:
    print(query)
    results = db_esml_e5_hybrid.similarity_search(query)
    [print(element.metadata) for element in results]
    add_result(search_logic="hybrid_elastic_bm25_e5", query=query, result=[element.metadata for element in results])

画像検索の方法について知りたい
{'source': 'https://www.elastic.co/jp/blog/5-technical-components-image-similarity-search'}
{'source': 'https://www.elastic.co/jp/blog/overview-image-similarity-search-in-elastic'}
{'source': 'https://www.elastic.co/jp/blog/implement-image-similarity-search-elastic'}
{'source': 'https://www.elastic.co/jp/blog/using-search-analytics-to-strengthen-ecommerce-solutions'}
検索速度を速くする方法について知りたい
{'source': 'https://www.elastic.co/jp/blog/how-to-personalize-search-experiences-using-elastic'}
{'source': 'https://www.elastic.co/jp/blog/5-technical-components-image-similarity-search'}
{'source': 'https://www.elastic.co/jp/blog/using-search-analytics-to-strengthen-ecommerce-solutions'}
{'source': 'https://www.elastic.co/jp/blog/how-to-build-scoped-search-suggestions-and-search-query-corrections'}
Webサイトに検索を設けるにはどうすればいいか
{'source': 'https://www.elastic.co/jp/blog/how-to-build-scoped-search-suggestions-and-search-query-corrections'}
{'source': 'https://www.elastic.co/jp/blog/using-sear

## 3-F. (HuggingFaceにある英語用モデルを利用） Elasticsearchのセマンティック検索

### インジェスト

In [282]:
from langchain.vectorstores.elasticsearch import ElasticsearchStore
from langchain.embeddings import HuggingFaceInferenceAPIEmbeddings

MODEL_ID="sentence-transformers/all-distilroberta-v1"

if not huggingface_api_Key:
    raise Exception("huggingface_api_Key is not set")

embedding = HuggingFaceInferenceAPIEmbeddings(
    api_key=huggingface_api_Key,
    model_name=MODEL_ID
)
db_huggingface_distilroberta = ElasticsearchStore(
    es_connection=es,
    index_name="test_index_huggingface_distilroberta",
    embedding=embedding
)


In [283]:
if db_huggingface_distilroberta.client.indices.exists(index="test_index_huggingface_distilroberta"):
    db_huggingface_distilroberta.client.delete_by_query(index="test_index_huggingface_distilroberta", body={"query": {"match_all": {}}})
db_huggingface_distilroberta.add_documents(docs)
db_huggingface_distilroberta.client.indices.refresh(index="test_index_huggingface_distilroberta")

ObjectApiResponse({'_shards': {'total': 2, 'successful': 1, 'failed': 0}})

### サーチ

In [329]:
for query in questions:
    print(query)
    results = db_huggingface_distilroberta.similarity_search(query)
    [print(element.metadata) for element in results]
    add_result(search_logic="vector_distilroberta", query=query, result=[element.metadata for element in results])

画像検索の方法について知りたい
{'source': 'https://www.elastic.co/jp/blog/overview-image-similarity-search-in-elastic'}
{'source': 'https://www.elastic.co/jp/blog/5-technical-components-image-similarity-search'}
{'source': 'https://www.elastic.co/jp/blog/using-search-analytics-to-strengthen-ecommerce-solutions'}
{'source': 'https://www.elastic.co/jp/blog/how-to-build-scoped-search-suggestions-and-search-query-corrections'}
検索速度を速くする方法について知りたい
{'source': 'https://www.elastic.co/jp/blog/overview-image-similarity-search-in-elastic'}
{'source': 'https://www.elastic.co/jp/blog/5-technical-components-image-similarity-search'}
{'source': 'https://www.elastic.co/jp/blog/6-ways-elastic-enterprise-search-creates-a-competitive-edge-in-ecommerce'}
{'source': 'https://www.elastic.co/jp/blog/using-search-analytics-to-strengthen-ecommerce-solutions'}
Webサイトに検索を設けるにはどうすればいいか
{'source': 'https://www.elastic.co/jp/blog/using-search-analytics-to-strengthen-ecommerce-solutions'}
{'source': 'https://www.elastic.co/jp/blo

## 3-G.（Elastic ELSERを利用） ElasticsearchcでSparse Vector検索

#### インジェスト

In [285]:
from langchain.vectorstores.elasticsearch import ElasticsearchStore
from langchain.embeddings import HuggingFaceInferenceAPIEmbeddings

db_elser = ElasticsearchStore(
    es_connection=es,
    index_name="test_index_elser",
    strategy=ElasticsearchStore.SparseVectorRetrievalStrategy()
)

In [331]:
if es.indices.exists(index="test_index_elser"):
    db_elser.client.delete_by_query(index="test_index_elser", body={"query": {"match_all": {}}})
db_elser.add_documents(docs)
db_elser.client.indices.refresh(index="test_index_elser")

ObjectApiResponse({'_shards': {'total': 2, 'successful': 2, 'failed': 0}})

#### サーチ

In [332]:
for query in questions:
    print(query)
    results = db_elser.similarity_search(query)
    [print(element.metadata) for element in results]
    add_result(search_logic="sparsevector_elastic_elser", query=query, result=[element.metadata for element in results])

画像検索の方法について知りたい
{'source': 'https://www.elastic.co/jp/blog/implement-image-similarity-search-elastic'}
{'source': 'https://www.elastic.co/jp/blog/how-to-build-scoped-search-suggestions-and-search-query-corrections'}
{'source': 'https://www.elastic.co/jp/blog/how-to-personalize-search-experiences-using-elastic'}
{'source': 'https://www.elastic.co/jp/blog/5-technical-components-image-similarity-search'}
検索速度を速くする方法について知りたい
{'source': 'https://www.elastic.co/jp/blog/how-to-build-scoped-search-suggestions-and-search-query-corrections'}
{'source': 'https://www.elastic.co/jp/blog/implement-image-similarity-search-elastic'}
{'source': 'https://www.elastic.co/jp/blog/how-to-personalize-search-experiences-using-elastic'}
{'source': 'https://www.elastic.co/jp/blog/6-ways-elastic-enterprise-search-creates-a-competitive-edge-in-ecommerce'}
Webサイトに検索を設けるにはどうすればいいか
{'source': 'https://www.elastic.co/jp/blog/using-spring-boot-with-elastic-app-search'}
{'source': 'https://www.elastic.co/jp/blog/buildi

# 4. RAG検索: LLMを組み合わせてみよう

## 4-1. RetrievalQA chainでQA回答をする

### 4-1-1. RetrievalQA chainのファンクションを作成

In [334]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI

def ask_llm(vectorstore, question, template=None):
    if not template:
        template = """Use the following pieces of context to answer the question at the end.
    If you don't know the answer, just say that you don't know, don't try to make up an answer.
    Use three sentences maximum and keep the answer as concise as possible.
    Always say "thanks for asking!" at the end of the answer.
    {context}
    Question: {question}
    Helpful Answer:"""
    QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

    llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
    qa_chain = RetrievalQA.from_chain_type(
        llm,
        chain_type="stuff",
        retriever=vectorstore.as_retriever(),
        chain_type_kwargs={"prompt": QA_CHAIN_PROMPT},
        return_source_documents=True
    )
    result = qa_chain({"query": question})
    pprint("------------ result -----------")
    pprint(result["result"])
    pprint("------------ source_documents -----------")
    pprint(result['source_documents'])
    return {"query": question, "result": f"{result['result']}\n{result['source_documents']}"}

### 4-1-2. 検索文書を文字数Splitする
様々なSplit方法を試してみます。

#### 4-1-2-A. RecursiveCharacterTextSplitterでSplitする

In [291]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

print("---- ALL DOCS----")
pprint(docs)
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 200)
all_splits = text_splitter.split_documents(docs)
print("---- SPLITTED first 10----")

for split in all_splits[:10]:
    pprint(split.metadata['source'])
    pprint(split.page_content)

---- ALL DOCS----
[Document(page_content='検索分析を使ってEコマースソリューションを強化する Eコマース企業のデータ可視化例 Elastic Cloudの無料トライアルに登録 著者  The Elastic Enterprise Search team Twitter リンクトイン Facebook メール 印刷 お客様にすばらしい体験を届けるためには、自社が使えるツールのすべてをしっかりと理解しておくことが重要です。特に、現在のように経済が不安定な時期に少しでも優位に立ちたいと思うのであれば、その重要性はいっそう大きくなると言えます。良質なカスタマーエクスペリエンスを提供できれば、ページを閲覧した人を商品の購入に導き、長期的な顧客に変えていくことができます。それこそが、他社に対して優位を得るための第一歩にほかなりません。  お客様に最適な体験を届けるために必要なのは、ビジネスデータだけではありません。自社のEコマースプラットフォームから集めた検索分析データも非常に重要です。お客様がサイトで検索機能を利用するたびに、お客様のニーズを把握するうえで重要なデータが供給されています。Elasticなら、そのようなデータからさまざまなインサイトを簡単に導き出せるため、製品ミックスの改善や、セールスの最適化に役立ちます。 お客様がサイトで製品やサービスを検索したまさにそのとき、サイトにはさまざまな情報が届いています。ユーザーエクスペリエンス自体についてだけでなく、その質についてもフィードバックが届いているのです。そのようなフィードバックを念頭に置いたソリューションを構築すれば、ユーザーに好印象を与え、エンゲージメントを高めることが可能になります。しかし、検索分析がもたらすインサイトは現在、十分に活用されているとは言えません。そのため、これをうまく利用していくことが、優れたユーザーエクスペリエンスの実現に向けた鍵となります。  重要なデータポイントの1つに、「検索回数が上位のクエリ」があります。上位のクエリを見ると、多くのお客様が強い関心を抱いている品目や、流行している品目が直接明らかになることがあるからです。このデータは特に、ユーザーの地理情報その他のプロファイル情報など、各種メタデータと合わせて検討すると有用な情報に変わります。たと

#### 4-1-2-B. NLTKTextSplitterでSplitする

In [192]:
%pip install -q nltk
from langchain.text_splitter import NLTKTextSplitter

print("---- ALL DOCS----")
pprint(docs)
text_splitter = NLTKTextSplitter(chunk_size = 500, chunk_overlap = 0)
all_splits = text_splitter.split_documents(docs)
print("---- SPLITTED first 10 ----")
[pprint(split.metadata['source'] + "\n" + split.page_content) for split in all_splits[:10]]

Note: you may need to restart the kernel to use updated packages.
---- ALL DOCS----
[Document(page_content='根本原因分析（RCA）とは何か？ | 包括的なRCAガイド  | ElasticSkip to main contentPlatformPlatformソリューションソリューション導入事例導入事例リソースリソース料金料金ドキュメントドキュメントLanguage pickerDeutschEnglishEspañolFrançais日本語한국어简体中文PortuguêsSearchLogin無料トライアルを始めるセールスへのお問い合わせElasticsearch Platformオブザーバビリティも、セキュリティも、検索ソリューションも、Elasticsearchプラットフォームならすべて実現できます。Elasticの概要ELK Stackデータインジェストから検索、分析、可視化まで、すべてが思いのままKibanaElasticsearch統合機能群ELK Stackの概要Elastic Cloudお好みのクラウドプロバイダーでElasticを活用し、必要な答えを見つけられます。Cloudの概要パートナーセールスへのお問い合わせオブザーバビリティアプリやインフラをまとめて可視化し、問題を先回りして修正できます。ログ監視アプリケーションパフォーマンス監視インフラ監視シンセティック監視リアルユーザー監視ユニバーサルプロファイリングAIOpsOpenTelemetryオブザーバビリティの概要セキュリティサイバー脅威を広範囲で防御、調査し迅速に対応できます。継続的な監視脅威ハンティング調査とインシデントレスポンス脅威保護の自動化セキュリティの概要Searchあらゆるクラウドで迅速に検索結果を示し、パーソナライゼーションも高められます。生成的AI検索アプリEコマースWebサイト社内コンテンツ検索カスタマーサポートSearchの概要業界別公的セクター金融サービス通信医療テクノロジー小売＆eコマース製造＆自動車すべての業界を表示ソリューション別次に取るべき行動を大規模かつ柔軟、スピーディに見つけられます。オブザーバビリティセキュリティSearchお客様事例Cisco社：

[None, None, None, None, None, None]

#### 4-1-2-C. CharacterTextSplitterでSplitする

In [290]:
from langchain.text_splitter import CharacterTextSplitter

print("---- All DOCS ----")
pprint(docs)
text_splitter = CharacterTextSplitter(separator = "\n\n", chunk_size = 500, chunk_overlap = 200, is_separator_regex=True)
all_splits = text_splitter.split_documents(docs)
print("---- SPLITTED first 10 ----")
for split in all_splits[:10]:
    pprint(split.metadata['source'])
    pprint(split.page_content)

---- All DOCS ----
[Document(page_content='検索分析を使ってEコマースソリューションを強化する Eコマース企業のデータ可視化例 Elastic Cloudの無料トライアルに登録 著者  The Elastic Enterprise Search team Twitter リンクトイン Facebook メール 印刷 お客様にすばらしい体験を届けるためには、自社が使えるツールのすべてをしっかりと理解しておくことが重要です。特に、現在のように経済が不安定な時期に少しでも優位に立ちたいと思うのであれば、その重要性はいっそう大きくなると言えます。良質なカスタマーエクスペリエンスを提供できれば、ページを閲覧した人を商品の購入に導き、長期的な顧客に変えていくことができます。それこそが、他社に対して優位を得るための第一歩にほかなりません。  お客様に最適な体験を届けるために必要なのは、ビジネスデータだけではありません。自社のEコマースプラットフォームから集めた検索分析データも非常に重要です。お客様がサイトで検索機能を利用するたびに、お客様のニーズを把握するうえで重要なデータが供給されています。Elasticなら、そのようなデータからさまざまなインサイトを簡単に導き出せるため、製品ミックスの改善や、セールスの最適化に役立ちます。 お客様がサイトで製品やサービスを検索したまさにそのとき、サイトにはさまざまな情報が届いています。ユーザーエクスペリエンス自体についてだけでなく、その質についてもフィードバックが届いているのです。そのようなフィードバックを念頭に置いたソリューションを構築すれば、ユーザーに好印象を与え、エンゲージメントを高めることが可能になります。しかし、検索分析がもたらすインサイトは現在、十分に活用されているとは言えません。そのため、これをうまく利用していくことが、優れたユーザーエクスペリエンスの実現に向けた鍵となります。  重要なデータポイントの1つに、「検索回数が上位のクエリ」があります。上位のクエリを見ると、多くのお客様が強い関心を抱いている品目や、流行している品目が直接明らかになることがあるからです。このデータは特に、ユーザーの地理情報その他のプロファイル情報など、各種メタデータと合わせて検討すると有用な情報に変わります。た

#### 4-1-2-D. HTMLHeaderTextSplitterでSplitする (うまくいかない)

In [297]:
%pip -q install lxml
%pip -q install markdownify

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [318]:
from langchain.document_loaders import AsyncHtmlLoader
from bs4 import BeautifulSoup
from pprint import pprint

def extract_main_content(html_content):
    soup = BeautifulSoup(html_content, "html.parser")
    main_html = soup.find("main")
    scripts = main_html.find_all('script')
    for s in scripts:
        s.extract()

    if main_html:
        return main_html.prettify()
    return None

# https://www.elastic.co/jp/sitemapから取得
urls = [
    "https://www.elastic.co/jp/what-is/root-cause-analysis",
    "https://www.elastic.co/jp/what-is/anomaly-detection",
    "https://www.elastic.co/jp/what-is/siem",
    "https://www.elastic.co/jp/what-is/opentelemetry",
    "https://www.elastic.co/jp/what-is/aiops",
    "https://www.elastic.co/jp/what-is/kubernetes-monitoring"
    ]
loader = AsyncHtmlLoader(urls) # Use to load raw html into page_content
html_docs = loader.load()

for html_doc in html_docs:
    pprint(html_doc.metadata)
    print("======= 抽出前 =======")
    pprint(html_doc.page_content[:1000])
    print("======= 抽出後 =======")
    html_doc.page_content = extract_main_content(html_doc.page_content)
    pprint(html_doc.page_content[:1000])
    print("==============")

Fetching pages: 100%|##########| 6/6 [00:00<00:00,  9.00it/s]

{'source': 'https://www.elastic.co/jp/what-is/root-cause-analysis'}
======= 抽出前 =======
('<!DOCTYPE html><html lang="ja-jp"><head><meta '
 'charSet="utf-8"/><title>根本原因分析（RCA）とは何か？ | 包括的なRCAガイド  | '
 'Elastic</title><link rel="apple-touch-icon" sizes="57x57" '
 'href="/apple-icon-57x57.png"/><link rel="apple-touch-icon" sizes="60x60" '
 'href="/apple-icon-60x60.png"/><link rel="apple-touch-icon" sizes="72x72" '
 'href="/apple-icon-72x72.png"/><link rel="apple-touch-icon" sizes="76x76" '
 'href="/apple-icon-76x76.png"/><link rel="apple-touch-icon" sizes="114x114" '
 'href="/apple-icon-114x114.png"/><link rel="apple-touch-icon" sizes="120x120" '
 'href="/apple-icon-120x120.pn/g"/><link rel="apple-touch-icon" '
 'sizes="144x144" href="/apple-icon-144x144.png"/><link rel="apple-touch-icon" '
 'sizes="152x152" href="/apple-icon-152x152.png"/><link rel="apple-touch-icon" '
 'sizes="180x180" href="/apple-icon-180x180.png"/><link rel="apple-touch-icon" '
 'sizes="192x192" href="/apple-icon-192

In [319]:
from langchain.text_splitter import HTMLHeaderTextSplitter

headers_to_split_on = [
    # ("h1", "Header 1"),
    # ("h2", "Header 2"),
    ("h3", "Header 3"),
    ("h4", "Header 4"),
]
all_html_splits = []
html_splitter = HTMLHeaderTextSplitter(headers_to_split_on=headers_to_split_on, return_each_element=False)
for html_doc in html_docs:
    html = f"<html><body>{html_doc.page_content}</body></html>"
    # pprint(BeautifulSoup(html, "html.parser").find_all("h3"))
    html_header_splits_of_1doc = html_splitter.split_text(html)
    print("---- HTML SPLITTED first 10 ----")
    for split in html_header_splits_of_1doc:
        pprint(split)
        # pprint(BeautifulSoup(split.page_content, "html.parser").find_all("div"))
        # if split.metadata.get("Header 3"):
        #     split.metadata['source'] = html_doc.metadata['source']
        #     pprint(split.metadata)
        #     # pprint(split.page_content[:100])
        #     # print("")
        #     all_html_splits.append(split)

---- HTML SPLITTED first 10 ----
Document(page_content='ソフトウェア開発における根本原因分析（RCA）とは何か？  \nElasticオブザーバビリティで、根本原因分析をすばやく実施  \n根本原因分析とは  \n根本原因分析（RCA）は、対症療法を試すのではなく、ソフトウェア開発チームが問題の核心から問題を特定して解決するために使用する、実績のあるトラブルシューティング手法です。 根本原因分析とは、構造化された段階的なプロセスであり、関連データを収集、分析し、それに対処する解決策をテストすることによって、主要な根本原因を探し出すように設計されています。  \n根本原因分析が重要である理由  \n根本原因分析がソフトウェア開発において不可欠なのは、体系的なアプローチによって、チームがより効率的にトラブルシューティングを行い、問題の再発を防ぐ長期的な解決策を開発できるからです。エラーや不具合の根本原因に対処することで、開発者はシステムの安定性、信頼性、効率性を確保し、コストのかかるダウンタイムを減らし、開発プロセスをスピードアップすることができます。また、RCAは開発者が問題の影響度と重大性に基づいて優先度を設定し、最も重大な問題に最初に取り組めるようにします。  \n根本原因分析を実行する方法  \n科学や工学から製造や医療に至るまで、あらゆる業種や分野で問題解決の手法として応用されている根本原因分析では、システムの欠陥や障害の根本的な要因を特定、理解するために、特定の一連のステップを実行する必要があります。ソフトウェア開発における根本原因分析の実施に関わるステップは、同じ普遍的なRCAの原則に従います。  \n手順1：問題を定義し、アラートを設定する（可能な場合） RCAの最初のステップは、問題を定義し、それを明確に理解することです。これには、アプリケーションの異常動作、システムパフォーマンスの低下、セキュリティインシデントなどの潜在的な問題を監視するためのアラートの設定が含まれます。 手順2．データを収集して分析し、潜在的な原因を判定する 問題が定義された後は、次のステップはデータの収集と分析です。これには、システムログ、アプリケーションパフォーマンスメトリック、ユーザーフィードバック、その他の関連するデー

In [262]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

chunk_size = 500
chunk_overlap = 30
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size, chunk_overlap=chunk_overlap
)

for split in all_html_splits[2:3]:
    pprint(split.metadata)
    pprint(split.page_content)

# Split
all_splits = text_splitter.split_documents(all_html_splits)
print("---- SPLITTED first 10 ----")
for split in all_splits:
    pprint(split.metadata)
    pprint(split.page_content)



{'source': 'https://www.elastic.co/jp/what-is/root-cause-analysis'}
('Search  \n'
 'Platform  \n'
 'Elasticsearch Platform  \n'
 'オブザーバビリティも、セキュリティも、検索ソリューションも、Elasticsearchプラットフォームならすべて実現できます。  \n'
 'Elasticの概要  \n'
 'ELK Stack  \n'
 'データインジェストから検索、分析、可視化まで、すべてが思いのまま  \n'
 'KibanaElasticsearch統合機能群  \n'
 'ELK Stackの概要  \n'
 'Elastic Cloud  \n'
 'お好みのクラウドプロバイダーでElasticを活用し、必要な答えを見つけられます。  \n'
 'Cloudの概要  \n'
 'パートナーセールスへのお問い合わせ  \n'
 'ソリューション  \n'
 'オブザーバビリティ  \n'
 'アプリやインフラをまとめて可視化し、問題を先回りして修正できます。  \n'
 'ログ監視アプリケーションパフォーマンス監視インフラ監視シンセティック監視リアルユーザー監視ユニバーサルプロファイリングAIOpsOpenTelemetry  \n'
 'オブザーバビリティの概要  \n'
 'セキュリティ  \n'
 'サイバー脅威を広範囲で防御、調査し迅速に対応できます。  \n'
 '継続的な監視脅威ハンティング調査とインシデントレスポンス脅威保護の自動化  \n'
 'セキュリティの概要  \n'
 'Search  \n'
 'あらゆるクラウドで迅速に検索結果を示し、パーソナライゼーションも高められます。  \n'
 '生成的AI検索アプリEコマースWebサイト社内コンテンツ検索カスタマーサポート  \n'
 'Searchの概要  \n'
 '業界別  \n'
 '公的セクター金融サービス通信医療テクノロジー小売＆eコマース製造＆自動車  \n'
 'すべての業界を表示  \n'
 '導入事例  \n'
 'ソリューション別  \n'
 '次に取るべき行動を大規模かつ柔軟、スピーディに見つけられます。  \n'
 'オブザーバビリティ

## 4-2. Retrieverを作成して、QAする

### 4-2-A. OpenAI Embedding

In [292]:
from langchain.vectorstores.elasticsearch import ElasticsearchStore
from langchain.embeddings.openai import OpenAIEmbeddings

embedding = OpenAIEmbeddings()

pprint(es.info())
db_split_openai = ElasticsearchStore(
    es_connection=es,
    index_name="test_split_index_openai",
    embedding=embedding
)

ObjectApiResponse({'name': 'instance-0000000024', 'cluster_name': '507a2cf6ba204071943512e0537eee58', 'cluster_uuid': 'oF-xDLtXRCet87gRuM3eJg', 'version': {'number': '8.10.2', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '6d20dd8ce62365be9b1aca96427de4622e970e9e', 'build_date': '2023-09-19T08:16:24.564900370Z', 'build_snapshot': False, 'lucene_version': '9.7.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})


#### インジェスト

In [293]:
if es.indices.exists(index="test_split_index_openai"):
    db_split_openai.client.delete_by_query(index="test_split_index_openai", body={"query": {"match_all": {}}})
db_split_openai.add_documents(all_splits)
db_split_openai.client.indices.refresh(index="test_split_index_openai")

ObjectApiResponse({'_shards': {'total': 2, 'successful': 2, 'failed': 0}})

In [294]:
for query in questions:
    print(query)
    results = db_split_openai.similarity_search(query)
    [print(element.metadata) for element in results]

画像検索の方法について知りたい
{'source': 'https://www.elastic.co/jp/blog/implement-image-similarity-search-elastic'}
{'source': 'https://www.elastic.co/jp/blog/implement-image-similarity-search-elastic'}
{'source': 'https://www.elastic.co/jp/blog/5-technical-components-image-similarity-search'}
{'source': 'https://www.elastic.co/jp/blog/overview-image-similarity-search-in-elastic'}
検索速度を速くする方法について知りたい
{'source': 'https://www.elastic.co/jp/blog/how-to-build-scoped-search-suggestions-and-search-query-corrections'}
{'source': 'https://www.elastic.co/jp/blog/how-to-build-scoped-search-suggestions-and-search-query-corrections'}
{'source': 'https://www.elastic.co/jp/blog/building-personalized-ecommerce-search-experiences-with-elastic'}
{'source': 'https://www.elastic.co/jp/blog/6-ways-elastic-enterprise-search-creates-a-competitive-edge-in-ecommerce'}
Webサイトに検索を設けるにはどうすればいいか
{'source': 'https://www.elastic.co/jp/blog/how-to-build-scoped-search-suggestions-and-search-query-corrections'}
{'source': 'https:/

#### RAGサーチ

In [335]:
for query in questions:
    print(query)
    result = ask_llm(vectorstore=db_split_openai, question=query)
    add_result(search_logic="rag_openai_embedding", query=query, result=result)


画像検索の方法について知りたい
'------------ result -----------'
('画像検索の方法は、アプリケーションが画像をベクトルに変換し、データセットで類似した画像を検索することです。具体的な手順は、3番目のタブの［類似した画像］に移動し、ディスクから画像をアップロードして、［検索］をクリックすることです。また、使用されているモデルは多言語で推論をサポートしているため、自分の言語で画像を検索することもできます。\n'
 'Thanks for asking!')
'------------ source_documents -----------'
[Document(page_content='画像をアップロードするだけでも、検索は動作します。アプリケーションは画像をベクトルに変換して、データセットで類似した画像を検索します。このためには、3番目のタブの［類似した画像］に移動し、ディスクから画像をアップロードして、［検索］をクリックします。 Elasticsearchで使用しているNLP（sentence-transformers/clip-ViT-B-32-multilingual-v1）モデルは多言語で、多数の言語での推論をサポートしているため、自分の言語で画像を検索してください。その後に、英語のテキストも使用して、結果を検証してください。 使用されているモデルが汎用モデルで、かなり精度が高いものの、得られる結果はユースケースや他の要因によって異なることに注意することが重要です。正解率を高める必要がある場合は、汎用モデルを適応させるか、独自のモデルを開発する必要があります。CLIPモデルは検索の出発点として提供されているだけのものです。 完全なコードはGitHub', metadata={'source': 'https://www.elastic.co/jp/blog/implement-image-similarity-search-elastic'}),
 Document(page_content='おめでとうございます！アプリケーションが起動して実行され、インターネットブラウザーを使用してhttp://127.0.0.1:5001からアクセスできます。 画像検索タブに移動して、画像を最も適切に説明するテキストを入力します。非

### 4-2-B. HuggingFace API: E5モデル

In [296]:
from langchain.vectorstores.elasticsearch import ElasticsearchStore
from langchain.embeddings import HuggingFaceInferenceAPIEmbeddings

if not huggingface_api_Key:
    raise Exception("huggingface_api_Key is not set")

MODEL_ID="intfloat/multilingual-e5-base"

embedding = HuggingFaceInferenceAPIEmbeddings(
    api_key=huggingface_api_Key,
    model_name=MODEL_ID
)
db_split_huggingface_e5 = ElasticsearchStore(
    es_connection=es,
    index_name="test_split_index_huggingface_e5",
    embedding=embedding
)

#### インジェスト

In [297]:
if es.indices.exists(index="test_split_index_huggingface_e5"):
    db_split_huggingface_e5.client.delete_by_query(index="test_split_index_huggingface_e5", body={"query": {"match_all": {}}})
db_split_huggingface_e5.add_documents(all_splits)
db_split_huggingface_e5.client.indices.refresh(index="test_split_index_huggingface_e5")

ObjectApiResponse({'_shards': {'total': 2, 'successful': 1, 'failed': 0}})

#### RAGサーチ

In [336]:
for query in questions:
    print(query)
    result = ask_llm(vectorstore=db_split_huggingface_e5, question=query)
    add_result(search_logic="rag_huggingface_e5", query=query, result=result)

画像検索の方法について知りたい
'------------ result -----------'
('画像検索の方法は、まず画像をベクトル化し、そのベクトルを使用して類似した画像を検出します。このプロセスは、機械学習モデルを使用して行われます。具体的な手法やアルゴリズムは文中には記載されていませんので、詳細は不明です。Thanks '
 'for asking!')
'------------ source_documents -----------'
[Document(page_content='環境が設定されたところで、次のステップを実行して、自然言語を使用して実際に画像を検索し、類似した画像を検出することができます。概念実証としてElasticが提供するFlaskアプリケーションを使用します。このWebアプリケーションにはシンプルなUIがあり、シンプルな方法で画像検索を行えます。このGitHub repoでプロトタイプFlaskアプリケーションを利用できます。\xa0 このアプリケーションはバックグラウンドで2つのタスクを実行します。検索文字列を検索ボックスに入力した後、機械学習_inferエンドポイントを使用してテキストがベクトル化されます。その後、ベクトルが格納されたインデックスのmy-image-embeddingsに対して、密ベクトルを含むクエリが実行されます。この例では、このようなクエリが2つあります。最初のAPI呼び出しは_inferエンドポイントを使用します。結果は密ベクトルです。 2番目のタスクの検索クエリでは、密ベクトルを利用して、スコア別にソートされた画像を取得します。', metadata={'source': 'https://www.elastic.co/jp/blog/implement-image-similarity-search-elastic'}),
 Document(page_content='すべての画像（写真）を「app/static/images」フォルダーに置きます。サブフォルダーまであるディレクトリ構造を使用して、常に画像が整理されるようにします。すべての画像の準備ができたら、数個のパラメーターを使用してスクリプトを実行します。 合理的な結果を得るためには、少なくとも200～300個の画像を準備することが非

### 4-2-C. Elastic MLのE5モデル

In [157]:
from langchain.vectorstores.elasticsearch import ElasticsearchStore
from langchain.embeddings.elasticsearch import ElasticsearchEmbeddings

MODEL_ID="intfloat__multilingual-e5-base"

embedding = ElasticsearchEmbeddings.from_es_connection(
    es_connection=es,
    model_id=MODEL_ID,
    input_field="text_field"
)

db_split_esml_e5 = ElasticsearchStore(
    es_connection=es,
    index_name="test_split_index_esml_e5",
    embedding=embedding,
    strategy=ElasticsearchStore.ApproxRetrievalStrategy()
)

#### インジェスト

In [299]:
if db_split_esml_e5.client.indices.exists(index="test_split_index_esml_e5"):
    db_split_esml_e5.client.delete_by_query(index="test_split_index_esml_e5", body={"query": {"match_all": {}}})

for split in all_splits:
    db_split_esml_e5.add_documents([split])
    print(".", end='')
    db_split_esml_e5.client.indices.refresh(index="test_split_index_esml_e5")

.................................................................................................................

#### RAGサーチ

In [337]:
for query in questions:
    print(query)
    result = ask_llm(vectorstore=db_split_esml_e5, question=query)
    add_result(search_logic="rag_elasticml_e5", query=query, result=result)

画像検索の方法について知りたい
'------------ result -----------'
('画像検索の方法は、Flaskアプリケーションを使用して行われます。アプリケーションにはシンプルなUIがあり、検索ボックスにテキストを入力することで画像を検索できます。また、画像をアップロードすることでも検索が可能です。  '
 'Thanks for asking!')
'------------ source_documents -----------'
[Document(page_content='すべての画像（写真）を「app/static/images」フォルダーに置きます。サブフォルダーまであるディレクトリ構造を使用して、常に画像が整理されるようにします。すべての画像の準備ができたら、数個のパラメーターを使用してスクリプトを実行します。 合理的な結果を得るためには、少なくとも200～300個の画像を準備することが非常に重要です。画像が少なすぎると、検索する空間が非常に小さく、検索ベクトルまでの距離も非常に短くなるため、想定した結果が得られません。image_embeddingsフォルダーでスクリプトを実行し、変数の値を使用します。 画像の数、サイズ、CPU、ネットワーク接続によっては、このタスクに少し時間がかかります。データセット全体を処理する前に、画像の数を少なくして実験してください。スクリプトが完了した後は、kibana devツールを使用して、インデックスmy-image-embeddingsが存在し、対応するドキュメントがあることを確認できます。', metadata={'source': 'https://www.elastic.co/jp/blog/implement-image-similarity-search-elastic'}),
 Document(page_content='環境が設定されたところで、次のステップを実行して、自然言語を使用して実際に画像を検索し、類似した画像を検出することができます。概念実証としてElasticが提供するFlaskアプリケーションを使用します。このWebアプリケーションにはシンプルなUIがあり、シンプルな方法で画像検索を行えます。このGitHub repoでプロトタイプFlaskアプリケーションを

### 4-2-D. Elastic RRFハイブリッド検索

In [301]:
from langchain.vectorstores.elasticsearch import ElasticsearchStore
from langchain.embeddings.elasticsearch import ElasticsearchEmbeddings

MODEL_ID="intfloat__multilingual-e5-base"

embedding = ElasticsearchEmbeddings.from_es_connection(
    es_connection=es,
    model_id=MODEL_ID,
    input_field="text_field"
)

db_split_esml_e5_hybrid = ElasticsearchStore(
    es_connection=es,
    index_name="test_split_index_esml_e5_hybrid",
    embedding=embedding,
    strategy=ElasticsearchStore.ApproxRetrievalStrategy(hybrid=True)
)

#### インジェスト

In [302]:
if db_split_esml_e5_hybrid.client.indices.exists(index="test_split_index_esml_e5_hybrid"):
    db_split_esml_e5_hybrid.client.delete_by_query(index="test_split_index_esml_e5_hybrid", body={"query": {"match_all": {}}})

for split in all_splits:
    db_split_esml_e5_hybrid.add_documents([split])
    print(".", end='')
    db_split_esml_e5_hybrid.client.indices.refresh(index="test_split_index_esml_e5_hybrid")

.................................................................................................................

#### RAGサーチ

In [338]:
for query in questions:
    print(query)
    result = ask_llm(vectorstore=db_split_esml_e5_hybrid, question=query)
    add_result(search_logic="rag_elasticml_hybrid_e5", query=query, result=result)

画像検索の方法について知りたい
'------------ result -----------'
('環境が設定された後、Flaskアプリケーションを使用して画像検索を行うことができます。このアプリケーションはシンプルなUIを持ち、自然言語を使用して類似した画像を検出することができます。GitHubのリポジトリでプロトタイプFlaskアプリケーションを利用できます。Thanks '
 'for asking!')
'------------ source_documents -----------'
[Document(page_content='環境が設定されたところで、次のステップを実行して、自然言語を使用して実際に画像を検索し、類似した画像を検出することができます。概念実証としてElasticが提供するFlaskアプリケーションを使用します。このWebアプリケーションにはシンプルなUIがあり、シンプルな方法で画像検索を行えます。このGitHub repoでプロトタイプFlaskアプリケーションを利用できます。\xa0 このアプリケーションはバックグラウンドで2つのタスクを実行します。検索文字列を検索ボックスに入力した後、機械学習_inferエンドポイントを使用してテキストがベクトル化されます。その後、ベクトルが格納されたインデックスのmy-image-embeddingsに対して、密ベクトルを含むクエリが実行されます。この例では、このようなクエリが2つあります。最初のAPI呼び出しは_inferエンドポイントを使用します。結果は密ベクトルです。 2番目のタスクの検索クエリでは、密ベクトルを利用して、スコア別にソートされた画像を取得します。', metadata={'source': 'https://www.elastic.co/jp/blog/implement-image-similarity-search-elastic'}),
 Document(page_content='すべての画像（写真）を「app/static/images」フォルダーに置きます。サブフォルダーまであるディレクトリ構造を使用して、常に画像が整理されるようにします。すべての画像の準備ができたら、数個のパラメーターを使用してスクリプトを実行します。 合理的な結果を得るためには、少

### 4-2-E. Elastic ELSER

In [304]:
from langchain.vectorstores.elasticsearch import ElasticsearchStore

db_split_elser = ElasticsearchStore(
    es_connection=es,
    index_name="test_split_index_elser",
    strategy=ElasticsearchStore.SparseVectorRetrievalStrategy()
)

#### インジェスト

In [305]:
if db_split_elser.client.indices.exists(index="test_split_index_elser"):
    db_split_elser.client.delete_by_query(index="test_split_index_elser", body={"query": {"match_all": {}}})

for split in all_splits:
    db_split_elser.add_documents([split])
    print(".", end='')
    db_split_elser.client.indices.refresh(index="test_split_index_elser")

.................................................................................................................

In [320]:
%pip install -q numpy
import numpy as np
# Define the size of sublists
chunk_size = 10

# Using numpy.array_split to split the list
split_list = np.array_split(all_splits, len(all_splits) // chunk_size)

if es.indices.exists(index="test_split_index_elser"):
    db_split_elser.client.delete_by_query(index="test_split_index_elser", body={"query": {"match_all": {}}})

for splits in split_list:
    db_split_elser.add_documents(splits)
    db_split_elser.client.indices.refresh(index="test_split_index_elser")

Note: you may need to restart the kernel to use updated packages.


#### RAGサーチ

In [339]:
for query in questions:
    print(query)
    result = ask_llm(vectorstore=db_split_elser, question=query)
    add_result(search_logic="rag_esml_elser", query=query, result=result)

画像検索の方法について知りたい
'------------ result -----------'
('この複数回にわたるブログシリーズでは、Elasticでプロトタイプの類似検索アプリケーションを開発する方法について説明しています。具体的には、画像の類似検索の5つの技術要素と、Elasticを使用して画像の類似検索を実装する方法について説明しています。詳細はブログシリーズを参照してください。Thanks '
 'for asking!')
'------------ source_documents -----------'
[Document(page_content='ページ全体を再読み込みするのではなく、結果の一部のみがサーバーによって返される内容で置換されます。この方法の利点は、JavaScriptを記述せずに実行できることです。これは、優れたhtmxライブラリによって可能になっています。記述を引用 > htmxでは、属性を使用して、直接HTMLでAJAX、CSS Transitions、WebSockets、サーバーによって送信されたイベントにアクセスできるため、ハイパーテキストのシンプルさと高機能を利用して最新のユーザーインターフェースを構築できます。 この例では、htmxの小さいサブセットのみを使用します。まず、2つのエンドポイント定義を見てみましょう。HTMLをレンダリングするエンドポイントと、必要なHTMLスニペットを返し、ページの一部のみを更新するエンドポイントです。 htmxでは、属性を使用して、直接HTMLでAJAX、CSS Transitions、WebSockets、サーバーによって送信されたイベントにアクセスできるため、ハイパーテキストのシンプルさと高機能を利用して最新のユーザーインターフェースを構築できます。', metadata={'source': 'https://www.elastic.co/jp/blog/using-spring-boot-with-elastic-app-search'}),
 Document(page_content='検索候補の絞り込みでは、特定のカテゴリやブランドに限定したクエリを候補として示すことで、ユーザーが検索を最も関心のあるトピックに絞り込むことができます。 これは、絞り込まれた結果セッ

# 5. より高度なRAGパターン

## 5-1. Self Query Retrieverで、質問文から検索のフィルターをかける

In [50]:
!python3 -m pip install -qU lark elasticsearch langchain openai

from langchain.schema import Document
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import ElasticsearchStore
from langchain.llms import OpenAI
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains.query_constructor.base import AttributeInfo

### 文書にタグづけする

In [370]:
metadatas = [
    #検索分析を使ってEコマースソリューションを強化する
    { "url": "https://www.elastic.co/jp/blog/using-search-analytics-to-strengthen-ecommerce-solutions", "tags": ["Eコマース","概要"] },

    #Elasticで、個々のお客様に応じたEコマース向け検索エクスペリエンスを構築
    { "url": "https://www.elastic.co/jp/blog/building-personalized-ecommerce-search-experiences-with-elastic", "tags": ["Eコマース", "概要"] },

    #Elastic App SearchでSpring Bootを使用する
    { "url": "https://www.elastic.co/jp/blog/using-spring-boot-with-elastic-app-search", "tags": ["実装方法"] },

    #Elasticエンタープライズサーチでeコマースの競争力を高める6つの方法
    { "url": "https://www.elastic.co/jp/blog/6-ways-elastic-enterprise-search-creates-a-competitive-edge-in-ecommerce", "tags": ["Eコマース","概要"] },

    #Elasticを使用して検索エクスペリエンスをパーソナライズする方法
    { "url": "https://www.elastic.co/jp/blog/how-to-personalize-search-experiences-using-elastic", "tags": ["Eコマース","概要"] },

    #検索候補の絞り込みと検索クエリの修正を実装する方法
    { "url": "https://www.elastic.co/jp/blog/how-to-build-scoped-search-suggestions-and-search-query-corrections", "tags": ["実装方法"] },

    #画像の類似検索の5つの技術要素
    { "url": "https://www.elastic.co/jp/blog/5-technical-components-image-similarity-search", "tags": ["画像検索","概要"] },

    #Elasticで画像の類似検索を実装する方法
    { "url": "https://www.elastic.co/jp/blog/implement-image-similarity-search-elastic", "tags": ["画像検索","実装方法"] },

    #Elasticsearchでの画像の類似検索の概要
    { "url": "https://www.elastic.co/jp/blog/overview-image-similarity-search-in-elastic", "tags": ["画像検索","概要"] },
]

In [408]:
for doc in docs:
    for metadata in metadatas:
        if doc.metadata['source'] == metadata['url']:
            doc.metadata['tags'] = metadata['tags']
pprint(docs[0])

Document(page_content='検索分析を使ってEコマースソリューションを強化する Eコマース企業のデータ可視化例 Elastic Cloudの無料トライアルに登録 著者  The Elastic Enterprise Search team Twitter リンクトイン Facebook メール 印刷 お客様にすばらしい体験を届けるためには、自社が使えるツールのすべてをしっかりと理解しておくことが重要です。特に、現在のように経済が不安定な時期に少しでも優位に立ちたいと思うのであれば、その重要性はいっそう大きくなると言えます。良質なカスタマーエクスペリエンスを提供できれば、ページを閲覧した人を商品の購入に導き、長期的な顧客に変えていくことができます。それこそが、他社に対して優位を得るための第一歩にほかなりません。  お客様に最適な体験を届けるために必要なのは、ビジネスデータだけではありません。自社のEコマースプラットフォームから集めた検索分析データも非常に重要です。お客様がサイトで検索機能を利用するたびに、お客様のニーズを把握するうえで重要なデータが供給されています。Elasticなら、そのようなデータからさまざまなインサイトを簡単に導き出せるため、製品ミックスの改善や、セールスの最適化に役立ちます。 お客様がサイトで製品やサービスを検索したまさにそのとき、サイトにはさまざまな情報が届いています。ユーザーエクスペリエンス自体についてだけでなく、その質についてもフィードバックが届いているのです。そのようなフィードバックを念頭に置いたソリューションを構築すれば、ユーザーに好印象を与え、エンゲージメントを高めることが可能になります。しかし、検索分析がもたらすインサイトは現在、十分に活用されているとは言えません。そのため、これをうまく利用していくことが、優れたユーザーエクスペリエンスの実現に向けた鍵となります。  重要なデータポイントの1つに、「検索回数が上位のクエリ」があります。上位のクエリを見ると、多くのお客様が強い関心を抱いている品目や、流行している品目が直接明らかになることがあるからです。このデータは特に、ユーザーの地理情報その他のプロファイル情報など、各種メタデータと合わせて検討すると有用な情報に変わります。たとえば、運営しているEコマースサイトの上

### インジェスト

In [389]:
from langchain.vectorstores.elasticsearch import ElasticsearchStore
from langchain.embeddings.elasticsearch import ElasticsearchEmbeddings

MODEL_ID="intfloat__multilingual-e5-base"

embedding = ElasticsearchEmbeddings.from_es_connection(
    es_connection=es,
    model_id=MODEL_ID
)

db_esml_e5_hybrid_tags = ElasticsearchStore(
    es_connection=es,
    index_name="test_index_esml_e5_hybrid_tags",
    embedding=embedding,
    strategy=ElasticsearchStore.ApproxRetrievalStrategy(hybrid=True)
)


In [390]:
if db_esml_e5_hybrid_tags.client.indices.exists(index="test_index_esml_e5_hybrid_tags"):
    db_esml_e5_hybrid_tags.client.delete_by_query(index="test_index_esml_e5_hybrid_tags", body={"query": {"match_all": {}}})
db_esml_e5_hybrid_tags.add_documents(docs)
db_esml_e5_hybrid_tags.client.indices.refresh(index="test_index_esml_e5_hybrid_tags")

ObjectApiResponse({'_shards': {'total': 2, 'successful': 1, 'failed': 0}})

### サーチ

In [401]:
# Add details about metadata fields
metadata_field_info = [
    AttributeInfo(
        name="tags",
        description="文書の特徴を表すタグ",
        type="string or list[string]",
    )
]

document_content_description = "Elasticsearchの検索に関するブログ。入門編か実装編かののタグが付与されている。特定のインダストリーに関するものはそのインダストリーのタグが付与されている。特定のソリューションに関するものはそのタグが付与されている。"

# Set up openAI llm with sampling temperature 0
llm = OpenAI(temperature=0)

# instantiate retriever
selfquery_retriever = SelfQueryRetriever.from_llm(
    llm, db_esml_e5_hybrid_tags, document_content_description, metadata_field_info, verbose=True
)

In [377]:
# Set logging for the queries
import logging

logging.basicConfig()
logging.getLogger("langchain.retrievers.self_query").setLevel(logging.INFO)

In [392]:
results = selfquery_retriever.get_relevant_documents("画像検索について知りたい")
[print(element.metadata) for element in results]

INFO:langchain.retrievers.self_query.base:Generated Query: query='画像検索' filter=None limit=None


{'source': 'https://www.elastic.co/jp/blog/5-technical-components-image-similarity-search', 'tags': ['画像検索', '概要']}
{'source': 'https://www.elastic.co/jp/blog/implement-image-similarity-search-elastic', 'tags': ['画像検索', '実装方法']}
{'source': 'https://www.elastic.co/jp/blog/overview-image-similarity-search-in-elastic', 'tags': ['画像検索', '概要']}
{'source': 'https://www.elastic.co/jp/blog/using-search-analytics-to-strengthen-ecommerce-solutions', 'tags': ['Eコマース', '概要']}


[None, None, None, None]

In [393]:
results = selfquery_retriever.get_relevant_documents("画像検索の詳しい実装方法について知りたい")
[print(element.metadata) for element in results]

INFO:langchain.retrievers.self_query.base:Generated Query: query='画像検索 実装' filter=Operation(operator=<Operator.AND: 'and'>, arguments=[Comparison(comparator=<Comparator.CONTAIN: 'contain'>, attribute='tags', value='実装編'), Comparison(comparator=<Comparator.CONTAIN: 'contain'>, attribute='tags', value='画像検索')]) limit=None


{'source': 'https://www.elastic.co/jp/blog/implement-image-similarity-search-elastic', 'tags': ['画像検索', '実装方法']}


[None]

In [394]:
results = selfquery_retriever.get_relevant_documents("Eコマースにおける画像検索について知りたい")
[print(element.metadata) for element in results]

INFO:langchain.retrievers.self_query.base:Generated Query: query='image search ecommerce' filter=Comparison(comparator=<Comparator.CONTAIN: 'contain'>, attribute='tags', value='e-commerce') limit=None


{'source': 'https://www.elastic.co/jp/blog/building-personalized-ecommerce-search-experiences-with-elastic', 'tags': ['Eコマース', '概要']}
{'source': 'https://www.elastic.co/jp/blog/how-to-personalize-search-experiences-using-elastic', 'tags': ['Eコマース', '概要']}
{'source': 'https://www.elastic.co/jp/blog/using-search-analytics-to-strengthen-ecommerce-solutions', 'tags': ['Eコマース', '概要']}
{'source': 'https://www.elastic.co/jp/blog/6-ways-elastic-enterprise-search-creates-a-competitive-edge-in-ecommerce', 'tags': ['Eコマース', '概要']}


[None, None, None, None]

In [395]:
results = selfquery_retriever.get_relevant_documents("Eコマースにおける画像検索の詳しい実装方法について知りたい")
[print(element.metadata) for element in results]

INFO:langchain.retrievers.self_query.base:Generated Query: query='image search ecommerce' filter=Operation(operator=<Operator.AND: 'and'>, arguments=[Comparison(comparator=<Comparator.CONTAIN: 'contain'>, attribute='tags', value='e-commerce'), Comparison(comparator=<Comparator.CONTAIN: 'contain'>, attribute='tags', value='implementation')]) limit=None


[]

### RAG用のSplitドキュメントにもタグづけする

In [409]:
for split in all_splits:
    for metadata in metadatas:
        if split.metadata['source'] == metadata['url']:
            split.metadata['tags'] = metadata['tags']
pprint(all_splits[0])
pprint(all_splits[1])

Document(page_content='検索分析を使ってEコマースソリューションを強化する Eコマース企業のデータ可視化例 Elastic Cloudの無料トライアルに登録 著者  The Elastic Enterprise Search team Twitter リンクトイン Facebook メール 印刷 お客様にすばらしい体験を届けるためには、自社が使えるツールのすべてをしっかりと理解しておくことが重要です。特に、現在のように経済が不安定な時期に少しでも優位に立ちたいと思うのであれば、その重要性はいっそう大きくなると言えます。良質なカスタマーエクスペリエンスを提供できれば、ページを閲覧した人を商品の購入に導き、長期的な顧客に変えていくことができます。それこそが、他社に対して優位を得るための第一歩にほかなりません。', metadata={'source': 'https://www.elastic.co/jp/blog/using-search-analytics-to-strengthen-ecommerce-solutions', 'tags': ['Eコマース', '概要']})
Document(page_content='お客様に最適な体験を届けるために必要なのは、ビジネスデータだけではありません。自社のEコマースプラットフォームから集めた検索分析データも非常に重要です。お客様がサイトで検索機能を利用するたびに、お客様のニーズを把握するうえで重要なデータが供給されています。Elasticなら、そのようなデータからさまざまなインサイトを簡単に導き出せるため、製品ミックスの改善や、セールスの最適化に役立ちます。 お客様がサイトで製品やサービスを検索したまさにそのとき、サイトにはさまざまな情報が届いています。ユーザーエクスペリエンス自体についてだけでなく、その質についてもフィードバックが届いているのです。そのようなフィードバックを念頭に置いたソリューションを構築すれば、ユーザーに好印象を与え、エンゲージメントを高めることが可能になります。しかし、検索分析がもたらすインサイトは現在、十分に活用されているとは言えません。そのため、これをうまく利用していくことが、優れたユーザーエクスペリエンスの実現に向けた鍵となります。', metadata={'sou

### RAG用のSplitデータをインジェスト

In [403]:
from langchain.vectorstores.elasticsearch import ElasticsearchStore
from langchain.embeddings.elasticsearch import ElasticsearchEmbeddings

MODEL_ID="intfloat__multilingual-e5-base"

embedding = ElasticsearchEmbeddings.from_es_connection(
    es_connection=es,
    model_id=MODEL_ID
)

db_split_esml_e5_hybrid_tags = ElasticsearchStore(
    es_connection=es,
    index_name="test_split_index_esml_e5_hybrid_tags",
    embedding=embedding,
    strategy=ElasticsearchStore.ApproxRetrievalStrategy(hybrid=True)
)

In [404]:
if db_split_esml_e5_hybrid_tags.client.indices.exists(index="test_split_index_esml_e5_hybrid_tags"):
    db_split_esml_e5_hybrid_tags.client.delete_by_query(index="test_split_index_esml_e5_hybrid_tags", body={"query": {"match_all": {}}})

for split in all_splits:
    db_split_esml_e5_hybrid_tags.add_documents([split])
    print(".", end='')
    db_split_esml_e5_hybrid_tags.client.indices.refresh(index="test_split_index_esml_e5_hybrid_tags")

.................................................................................................................

### RAGでのサーチ

In [405]:
# Add details about metadata fields
metadata_field_info = [
    AttributeInfo(
        name="tags",
        description="文書の特徴を表すタグ",
        type="string or list[string]",
    )
]

document_content_description = "Elasticsearchの検索に関するブログ。入門編か実装編かののタグが付与されている。特定のインダストリーに関するものはそのインダストリーのタグが付与されている。特定のソリューションに関するものはそのタグが付与されている。"

# Set up openAI llm with sampling temperature 0
llm = OpenAI(temperature=0)

# instantiate retriever
selfquery_retriever = SelfQueryRetriever.from_llm(
    llm, db_split_esml_e5_hybrid_tags, document_content_description, metadata_field_info, verbose=True
)

In [406]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI

def ask_llm_with_selfquery(selfquery_retriever, question, template=None):
    if not template:
        template = """Use the following pieces of context to answer the question at the end.
    If you don't know the answer, just say that you don't know, don't try to make up an answer.
    Use three sentences maximum and keep the answer as concise as possible.
    Always say "thanks for asking!" at the end of the answer.
    {context}
    Question: {question}
    Helpful Answer:"""
    QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

    llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
    qa_chain = RetrievalQA.from_chain_type(
        llm,
        chain_type="stuff",
        retriever=selfquery_retriever,
        chain_type_kwargs={"prompt": QA_CHAIN_PROMPT},
        return_source_documents=True
    )
    result = qa_chain({"query": question})
    pprint("------------ result -----------")
    pprint(result["result"])
    pprint("------------ source_documents -----------")
    pprint(result['source_documents'])
    return {"query": question, "result": f"{result['result']}\n{result['source_documents']}"}

In [407]:
query="画像検索の詳しい実装方法について知りたい"
print(query)
result = ask_llm_with_selfquery(selfquery_retriever=selfquery_retriever, question=query)
add_result(search_logic="rag_esml_e5_hybrid_selfquery", query=query, result=result)


画像検索の詳しい実装方法について知りたい


INFO:langchain.retrievers.self_query.base:Generated Query: query='画像検索 実装' filter=Operation(operator=<Operator.AND: 'and'>, arguments=[Comparison(comparator=<Comparator.CONTAIN: 'contain'>, attribute='tags', value='実装編'), Comparison(comparator=<Comparator.CONTAIN: 'contain'>, attribute='tags', value='画像検索')]) limit=None


'------------ result -----------'
('画像検索の詳細な実装方法については、提供された文脈では具体的な情報が提供されていません。申し訳ありませんが、詳細な実装方法についてはわかりません。Thanks '
 'for asking!')
'------------ source_documents -----------'
[Document(page_content='すべての画像（写真）を「app/static/images」フォルダーに置きます。サブフォルダーまであるディレクトリ構造を使用して、常に画像が整理されるようにします。すべての画像の準備ができたら、数個のパラメーターを使用してスクリプトを実行します。 合理的な結果を得るためには、少なくとも200～300個の画像を準備することが非常に重要です。画像が少なすぎると、検索する空間が非常に小さく、検索ベクトルまでの距離も非常に短くなるため、想定した結果が得られません。image_embeddingsフォルダーでスクリプトを実行し、変数の値を使用します。 画像の数、サイズ、CPU、ネットワーク接続によっては、このタスクに少し時間がかかります。データセット全体を処理する前に、画像の数を少なくして実験してください。スクリプトが完了した後は、kibana devツールを使用して、インデックスmy-image-embeddingsが存在し、対応するドキュメントがあることを確認できます。', metadata={'source': 'https://www.elastic.co/jp/blog/implement-image-similarity-search-elastic', 'tags': ['画像検索', '実装方法']}),
 Document(page_content='おめでとうございます！アプリケーションが起動して実行され、インターネットブラウザーを使用してhttp://127.0.0.1:5001からアクセスできます。 画像検索タブに移動して、画像を最も適切に説明するテキストを入力します。非キーワードまたは説明テキストを使用してみてください。次の例では、「endless route to the top」というテキストが入力されました。 結果はデータセットから表示されます。

## 5-2. Agentsを使い、外部プログラムとしてElasticsearchのデータを検索させる

In [410]:
# Import things that are needed generically
from langchain.chains import LLMMathChain
from langchain.utilities import SerpAPIWrapper
from langchain.agents import AgentType, initialize_agent
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool, StructuredTool, Tool, tool

In [411]:
llm = ChatOpenAI(temperature=0)

In [412]:
from langchain.tools import tool

@tool
def search_api(query: str) -> str:
    """Searches the API for the query."""
    return f"Results for query {query}"

In [465]:
from langchain.tools import tool


@tool
def elasticsearch_doc_counts_api(query: str) -> str:
    """Searches the API for the query."""
    query = {
        "bool": {
            "must": [
                {
                    "match": {
                        "page_content.kuromoji": {
                            "query": query,
                            "analyzer": "ja_kuromoji_search_analyzer"
                        }
                    }
                }
            ]
        }
    }
    fields = ["metadata", "page_content"]
    response = es.search(index=INDEX_NAME, query=query, fields=fields, size=10)

    # Iterate through the search results and access the fields you want
    for hit in response['hits']['hits']:
        metadata = hit['_source']['metadata']
        page_content = hit['_source']['page_content']
        score = hit['_score']
        print(f"_score: {score}, metadata: {metadata}")
    
    return len(response['hits']['hits'])


In [466]:
from langchain.tools import tool

def vector_doc_counts_api_impl(vectorstore, query):
    results = vectorstore.similarity_search(query, 50)
    source_doc_count = set() 
    for element in results:
        print(element.metadata['source'])
        source_doc_count.add(element.metadata['source'])

    return len(source_doc_count)

@tool
def vector_doc_counts_api(query: str) -> str:
    """Searches the API for the query."""
    return vector_doc_counts_api_impl(vectorstore=db_split_openai, query=query)

In [467]:
# Load the tool configs that are needed.
tools = [
    Tool.from_function(
        func=vector_doc_counts_api.run,
        name="Search with vector search",
        description="ベクトル検索でクエリーして確認できた検索対象の文書の数を返すAPI",
    ),
    Tool.from_function(
        func=elasticsearch_doc_counts_api.run,
        name="Elasticsearch",
        description="Elasticsearchでクエリーして確認できた検索対象の文書の数を返すAPI",
    )
    ]

In [468]:
# Construct the agent. We will use the default agent type here.
# See documentation for a full list of options.
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [469]:
agent.run(
    "Eコマースに関する文書は何件ありますか?"
)



> Entering new AgentExecutor chain...
I need to find the number of documents related to E-commerce.
Action: Search with vector search
Action Input: "Eコマース"https://www.elastic.co/jp/blog/building-personalized-ecommerce-search-experiences-with-elastic
https://www.elastic.co/jp/blog/6-ways-elastic-enterprise-search-creates-a-competitive-edge-in-ecommerce
https://www.elastic.co/jp/blog/6-ways-elastic-enterprise-search-creates-a-competitive-edge-in-ecommerce
https://www.elastic.co/jp/blog/building-personalized-ecommerce-search-experiences-with-elastic
https://www.elastic.co/jp/blog/using-search-analytics-to-strengthen-ecommerce-solutions
https://www.elastic.co/jp/blog/building-personalized-ecommerce-search-experiences-with-elastic
https://www.elastic.co/jp/blog/6-ways-elastic-enterprise-search-creates-a-competitive-edge-in-ecommerce
https://www.elastic.co/jp/blog/building-personalized-ecommerce-search-experiences-with-elastic
https://www.elastic.co/jp/blog/how-to-build-scoped-search-sugge

'There are 8 documents related to E-commerce.'

In [470]:
agent.run(
    "Eコマースに関する文書は何件ありますか？全ての検索方法での結果をそれぞれ教えてください"
)



> Entering new AgentExecutor chain...
I need to find the number of documents related to E-commerce using both vector search and Elasticsearch.
Action: Search with vector search
Action Input: "Eコマース"https://www.elastic.co/jp/blog/building-personalized-ecommerce-search-experiences-with-elastic
https://www.elastic.co/jp/blog/6-ways-elastic-enterprise-search-creates-a-competitive-edge-in-ecommerce
https://www.elastic.co/jp/blog/6-ways-elastic-enterprise-search-creates-a-competitive-edge-in-ecommerce
https://www.elastic.co/jp/blog/using-search-analytics-to-strengthen-ecommerce-solutions
https://www.elastic.co/jp/blog/building-personalized-ecommerce-search-experiences-with-elastic
https://www.elastic.co/jp/blog/building-personalized-ecommerce-search-experiences-with-elastic
https://www.elastic.co/jp/blog/6-ways-elastic-enterprise-search-creates-a-competitive-edge-in-ecommerce
https://www.elastic.co/jp/blog/building-personalized-ecommerce-search-experiences-with-elastic
https://www.elastic.

'There are 8 documents related to E-commerce using vector search and 5 documents related to E-commerce using Elasticsearch.'

# 結果まとめ

In [367]:
for query in questions:
    print(f"query: {query}")
    print("")
    for result in result_list:
        if result['query'] == query:
            print(f"search_logic: {result['search_logic']}")
            pprint(f"{result['result']}")
            print('---------------------------')
    print('==============================')

query: 画像検索の方法について知りたい

search_logic: bm25
('{\'query\': \'画像検索の方法について知りたい\', \'result\': [\'{"metadata": {"source": '
 '"https://www.elastic.co/jp/blog/6-ways-elastic-enterprise-search-creates-a-competitive-edge-in-ecommerce"}, '
 '"highlight": {"page_content.kuromoji": '
 '["Elasticエンタープライズサーチでeコマースの競争力を高める6つの<em>方法</em> Elastic Cloudの無料トライアルに登録 '
 'お使いの<em>検索</em>アプリケーションは、皆さんが思う以上にパワフルです。", '
 '"今回ご紹介する機能を使えば、<em>検索</em>データを活用して優れた顧客エクスペリエンスを構築できます。", '
 '"「たとえば、皆さんが会議に加わって『ユーザーがクエリXの後に何を<em>検索</em>しているか<em>知り</em>たい』と言ったときに、ユーザーが<em>検索</em>するキーワードのシーケンスを作成できます。", '
 '"コンバージョンと、コンバージョンに至る前の<em>検索</em>で区切ることも可能です」 '
 '<em>検索</em>と発見は重要ですが、独立して運用されているわけではありません。", '
 '"バックログには膨大な数の商品があり、それを最初の<em>検索</em>結果に表示させたい場合もあるでしょう。"]}}\', '
 '\'{"metadata": {"source": '
 '"https://www.elastic.co/jp/blog/5-technical-components-image-similarity-search"}, '
 '"highlight": {"page_content.kuromoji": ["<em>画像</em>の類似<em>検索</em>の5つの技術要素 '
 'AI時代の<em>検索</em>ツールキット 推論：ユーザーのクエリをベクトル化する  <em>検索</em>：類似<e